In [52]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/us-patent-phrase-to-phrase-matching/sample_submission.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv


In [53]:
from fastai.imports import *

In [54]:
df=pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv')

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36473 entries, 0 to 36472
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       36473 non-null  object 
 1   anchor   36473 non-null  object 
 2   target   36473 non-null  object 
 3   context  36473 non-null  object 
 4   score    36473 non-null  float64
dtypes: float64(1), object(4)
memory usage: 1.4+ MB


In [56]:
df.describe(include=object)

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,8d135da0b55b8c88,component composite coating,composition,H01
freq,1,152,24,2186


In [57]:
df.describe()

,score
count,36473.000000
mean,0.362062
std,0.258335
min,0.000000
25%,0.250000
50%,0.250000
75%,0.500000
max,1.000000


In [58]:
df.rename(columns={'score': 'label'}, inplace=True)


In [59]:
df

,id,anchor,target,context,label
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [60]:
## cleaning / transofrmation
df.shape

(36473, 5)

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36473 entries, 0 to 36472
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       36473 non-null  object 
 1   anchor   36473 non-null  object 
 2   target   36473 non-null  object 
 3   context  36473 non-null  object 
 4   label    36473 non-null  float64
dtypes: float64(1), object(4)
memory usage: 1.4+ MB


In [62]:
df.duplicated().sum()

0

In [63]:
df.describe(include=object)


,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,8d135da0b55b8c88,component composite coating,composition,H01
freq,1,152,24,2186


In [64]:

df.con=df['context'].str[0]

/tmp/ipykernel_36/1889200089.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.con=df['context'].str[0]


In [65]:
df.con

0        A
1        A
2        A
3        A
4        A
        ..
36468    B
36469    B
36470    B
36471    B
36472    B
Name: context, Length: 36473, dtype: object

In [66]:
df

,id,anchor,target,context,label
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [67]:

df['input']=df.anchor + '; ' + df.target+'; '+df.con

In [68]:
df.input

0        abatement; abatement of pollution; A
1                abatement; act of abating; A
2               abatement; active catalyst; A
3           abatement; eliminating process; A
4                 abatement; forest region; A
                         ...                 
36468         wood article; wooden article; B
36469             wood article; wooden box; B
36470          wood article; wooden handle; B
36471        wood article; wooden material; B
36472       wood article; wooden substrate; B
Name: input, Length: 36473, dtype: object

In [69]:
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [70]:
import transformers

In [71]:
from transformers import AutoTokenizer

In [72]:
model='AI-Growth-Lab/PatentSBERTa'

In [73]:
tkz=AutoTokenizer.from_pretrained(model)

In [74]:
def tokz_f(x):
    return tkz(x['input'])

In [75]:
!pip install Dataset

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [76]:
from datasets import Dataset

In [77]:
ds=Dataset.from_pandas(df)

In [78]:
tokz_f(ds[0])

{'input_ids': [0, 19561, 18536, 4769, 1029, 19561, 18536, 4769, 2001, 10800, 1029, 1041, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [79]:
ds=ds.rename_column('label','score')

In [80]:
tokz_ds=ds.map(tokz_f,batched=True)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [81]:
tokz_ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input', 'input_ids', 'attention_mask'],
    num_rows: 36473
})

In [82]:
tokz_ds=tokz_ds.remove_columns(['id', 'anchor', 'target', 'context', 'input'])

In [83]:
def corr(x,y): return np.corrcoef(x,y)[0][1]


In [84]:
##Metrics
def corr(x,y): return np.corrcoef(x,y)[0][1]
def corr_co(eval_pred):
    return {'pearson':corr(*eval_pred)}

In [85]:
tokz_ds=tokz_ds.rename_columns({'score':'labels'})

In [86]:
tokz_ds

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 36473
})

In [87]:
##Splitting fo data
from sklearn.model_selection import train_test_split
a=[1,2,3,4,5]

train, test = train_test_split(a, test_size=0.3, )
train,test

([3, 1, 4], [2, 5])

In [88]:
##Splitting data 
from sklearn.model_selection import train_test_split
train_ds,test_ds=train_test_split(tokz_ds,test_size=0.3)

In [89]:
split_ds = tokz_ds.train_test_split(test_size=0.2)
split_ds

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 29178
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 7295
    })
})

In [90]:
from transformers import AutoModelForSequenceClassification

In [91]:
clss_model=AutoModelForSequenceClassification.from_pretrained(model,num_labels=1)

Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at AI-Growth-Lab/PatentSBERTa and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [103]:
#def corr_co(eval_pred):
#    return {'pearson': np.corrcoef(*eval_pred)[0,1]}


In [113]:
import numpy as np

def pearson_metric(eval_pred):
    preds, labels = eval_pred
    preds = preds.squeeze()  # remove extra dimensions if any
    labels = labels.squeeze()
    if preds.shape != labels.shape:
        # If shapes don't match, try flattening further
        preds = preds.flatten()
        labels = labels.flatten()
    # Compute Pearson correlation coefficient using np.corrcoef
    corr_matrix = np.corrcoef(preds, labels)
    pearson_corr = corr_matrix[0, 1]
    # Handle case where correlation might be NaN (e.g., zero variance)
    if np.isnan(pearson_corr):
        pearson_corr = 0.0
    return {'pearson': pearson_corr}


In [92]:
from transformers import Trainer,TrainingArguments

In [93]:
lr=8e-5
bs=128
wd=0.01
epochs=4

In [94]:
args=TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
        eval_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
        num_train_epochs=epochs, weight_decay=wd, report_to='none')

In [114]:
trainer=Trainer(clss_model, args, train_dataset=split_ds['train'],eval_dataset=split_ds['test'],
                  tokenizer=tkz, compute_metrics=pearson_metric)

/tmp/ipykernel_36/3714226766.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(clss_model, args, train_dataset=split_ds['train'],eval_dataset=split_ds['test'],


In [115]:
trainer.train()

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.024398,0.816693
2,No log,0.023841,0.828204
3,0.003800,0.024018,0.832435
4,0.003800,0.023484,0.832681


TrainOutput(global_step=912, training_loss=0.0039035806007552565, metrics={'train_runtime': 252.9713, 'train_samples_per_second': 461.365, 'train_steps_per_second': 3.605, 'total_flos': 1175883109629516.0, 'train_loss': 0.0039035806007552565, 'epoch': 4.0})

In [116]:
eval_df=pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv')

In [118]:
eval_df.head()

,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04


In [119]:

eval_df.con=eval_df['context'].str[0]
eval_df['input']=eval_df.anchor + '; ' + eval_df.target+'; '+eval_df.con


/tmp/ipykernel_36/361381549.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  eval_df.con=eval_df['context'].str[0]


In [122]:
eval_df.head()

,id,anchor,target,context,input
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,opc drum; inorganic photoconductor drum; G
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,adjust gas flow; altering gas flow; F
2,36baf228038e314b,lower trunnion,lower locating,B60,lower trunnion; lower locating; B
3,1f37ead645e7f0c8,cap component,upper portion,D06,cap component; upper portion; D
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,neural stimulation; artificial neural network; H


In [123]:
eval_ds=Dataset.from_pandas(eval_df)

In [124]:
eval_tokz=eval_ds.map(tokz_f,batched=True)

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [127]:
eval_tokz=eval_tokz.remove_columns(['id','anchor', 'target', 'context', 'input',])

In [130]:
pred=trainer.predict(eval_tokz).predictions.astype(float)

In [132]:
preds = np.clip(pred, 0, 1)


In [137]:
import datasets

submission = datasets.Dataset.from_dict({
    'id': eval_ds['id'],
    'score': preds
})

submission.to_csv('submission.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1007